# Medium claps predictor

## Idea

In [1]:
import os
import pandas as pd
#import torch 
#import torchtext
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random
import numpy as np
%matplotlib inline

In [2]:
DATA_PATH = './data'
TRAIN_PATH = os.path.join(DATA_PATH, 'articles_train.csv')
TEST_PATH = os.path.join(DATA_PATH, 'articles_test.csv')
SCRAPPING_OUT_PATH = os.path.join('./', 'scrapped_data')
MORE_PATH = os.path.join(DATA_PATH, "medium_data.csv")


In [3]:
# #За детерминизм!
# SEED = 0xDEAD
# random.seed(SEED)#
# np.random.seed(SEED)
# torch.random.manual_seed(SEED)
# torch.cuda.random.manual_seed_all(SEED)

In [4]:
# torch.cuda.is_available()

## Data Preparation

In [5]:
train_data = pd.read_csv(TRAIN_PATH, index_col='id')
test_data = pd.read_csv(TEST_PATH, index_col="id")
another_data = pd.read_csv(MORE_PATH, index_col="id")

In [6]:
train_data.shape, test_data.shape, another_data.shape

((3756, 6), (500, 5), (6508, 9))

In [7]:
another_data.head()

,url,title,subtitle,image,claps,responses,reading_time,publication,date
id,,,,,,,,,
1,https://towardsdatascience.com/a-beginners-gui...,A Beginner’s Guide to Word Embedding with Gens...,NaN,1.png,850,8,8,Towards Data Science,2019-05-30
2,https://towardsdatascience.com/hands-on-graph-...,Hands-on Graph Neural Networks with PyTorch & ...,NaN,2.png,1100,11,9,Towards Data Science,2019-05-30
3,https://towardsdatascience.com/how-to-use-ggpl...,How to Use ggplot2 in Python,A Grammar of Graphics for Python,3.png,767,1,5,Towards Data Science,2019-05-30
4,https://towardsdatascience.com/databricks-how-...,Databricks: How to Save Files in CSV on Your L...,When I work on Python projects dealing…,4.jpeg,354,0,4,Towards Data Science,2019-05-30
5,https://towardsdatascience.com/a-step-by-step-...,A Step-by-Step Implementation of Gradient Desc...,One example of building neural…,5.jpeg,211,3,4,Towards Data Science,2019-05-30


In [8]:
train_data.head(3)

,author,reading_time,link,title,text,claps
id,,,,,,
0,Joseph Rocca,23,https://towardsdatascience.com/understanding-v...,Understanding Variational Autoencoders (VAEs) ...,"Towards Data Science\nSep 24, 2019\nThis post ...",8300.0
1,Guido Vivaldi,6,https://towardsdatascience.com/using-mixed-eff...,Using Mixed-Effects Models For Linear Regressi...,"Towards Data Science\nMay 17, 2019\nMixed-effe...",336.0
2,Jerry Chen,11,https://news.greylock.com/the-new-moats-53f61a...,The New Moats. Why Systems of IntelligenceTM a...,"Greylock Perspectives\nApr 24, 2017\nTo build ...",3500.0


## Feature Engineering

In [9]:

# ! title clastering (maybe topic modeling LDA)
# ! link parser (some sites are more popular than another)
# ? text parser (only text without titles/dates...)
# ! link scrapper (img per 1000 words, image sizes, count of words)
# ! catboost, lgbm, ridge, random_forest

## Article Scrapper

In [10]:
import requests
import re
import time
import pandas as pd
import numpy as np
import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

def is_English(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

def clean_string(a_string):
    new_string = re.sub(r'‘|’|"|—|“|”|,', '', a_string).strip()
    new_string = re.sub(r'–', ' ', new_string)
    #new_string = re.sub(r'é', 'e', new_string)
    return new_string

def filter_string(a_string):
    if a_string == '.':
        return False
    elif a_string == '':
        return False
    else:
        return True

In [11]:
import re

class ArticleScraper():
    """
    Scrapes all data for an article.
    """
    def __init__(self):
        self.scraper_class = 'ArticleScraper' 

    
    def get_title(self, soup):
        try:
            title = soup.find('h1').text
            return title
        
        except:
            print("Couldn't get title from article.")

    
    def get_subtitle(self, soup):
        try:
            return soup.find('section').find('h2').text
        except:
            print("Couldn't get subtitle from article.")


    def get_publication(self, soup):
        try:
            for div in soup.find_all("div"):
                if div.text == 'Published in':
                    try:
                        return div.next_element.next_element.next_element.find('p').text
                    except:
                        continue
        except:
            print("Couldn't get publication type from article.")
            return "Medium"


    def get_author(self, soup):
        try:
            return soup.find('h2', class_=r"pw-author-name").text
        except:
            print("Couldn't get title from article.") 


    def get_reading_time(self, soup):
        try:
            return int(soup.find('div', class_=r"pw-reading-time").text.split()[0])
        except:
            print("Couldn't get reading time from article.")

    def get_claps(self, soup):
        try:
            claps = soup.find_all("div", {"class": "pw-multi-vote-count"})[0].text
            tens = {'K': 10e2, 'M': 10e5, 'B': 10e8, 'k': 10e2, 'm': 10e5, 'b': 10e8}
            if (claps[-1] != 'K' and claps[-1] != 'M' 
                        and claps[-1] != 'k' and claps[-1] != 'm'
                        and claps[-1] != 'b' and claps[-1] != 'B'):
                        return int(claps)
            f = lambda x: int(float(x[:-1])*tens[x[-1]])
            return f(claps)
        except: 
            print("Can't get claps information")


    def get_responses(self, soup): 
        try:
            responses = [x.text for x in soup.find_all("span") if x.has_attr("class") and 'pw-responses-count' in x['class']][0].split()[0]
            tens = {'K': 10e2, 'M': 10e5, 'B': 10e8, 'k': 10e2, 'm': 10e5, 'b': 10e8}
            if (responses[-1] != 'K' and responses[-1] != 'M' 
                        and responses[-1] != 'k' and responses[-1] != 'm'
                        and responses[-1] != 'b' and responses[-1] != 'B'):
                        return int(responses)
            f = lambda x: int(float(x[:-1])*tens[x[-1]])
            return f(responses)
        except:
            print("Can't get responses information")


    def get_date(self, soup):
        try:
            date_string = soup.find('p', class_=r"pw-published-date").text 
            return datetime.datetime.strptime(date_string, '%b %d, %Y').strftime('%d/%m/%Y')
        except:
            print("Couldn't get date from article.") 

    def get_followers(self, soup):
        try:
            
            fol_string = [x.text for x in soup.find_all("span") if x.has_attr("class") and 'pw-follower-count' in x['class']][0].split()[0]
            tens = {'K': 10e2, 'M': 10e5, 'B': 10e8, 'k': 10e2, 'm': 10e5, 'b': 10e8}
            if (fol_string[-1] != 'K' and fol_string[-1] != 'M' 
                        and fol_string[-1] != 'k' and fol_string[-1] != 'm'
                        and fol_string[-1] != 'b' and fol_string[-1] != 'B'):
                        return int(fol_string)
            f = lambda x: int(float(x[:-1])*tens[x[-1]])
            return f(fol_string)
        except:
            print("Can't go to the followers info")    

    
    def get_mean_size(self, soup):
        try:
            pics = soup.find('section').find_all('img')#, width=True, height=True)
            sums = (0,0)
            for pic in pics:
                url = pic.get('src')
                im = Image.open(requests.get(url, stream=True).raw)
                sums = tuple(map(sum, zip(sums, im.size)))
            mean = tuple(ti//len(pics) for ti in sums)
            if len(pics) == 0:
                return (0,0)
            return mean
        except:
            print("Can't get image's sizes")
         

    def count_figures(self, soup):
        try:
            return len(soup.find('section').find_all('img'))
        except:
            print("Can't get amount of pictures")

    def get_pure_text(self, soup):
        try:
            pure_text = ''
            for unparsed in soup.find('section').find_all('p'):
                pure_text += unparsed.text
            return pure_text
        except:
            print("Can't get pure text")
    
    
    def count_words(self, soup):
        try:
           pure_text = self.get_pure_text(soup=soup)
           return len(pure_text.split())
        except:
            print("Can't get words count from the article")

    
    def count_lists(self, soup):
        try:
            return len(soup.find('section').find_all('ol')) + len(soup.find('section').find_all('ul'))
        except:
            print("Can't get lists count")
    
    def bold_text_count(self, soup):
        try:
            return len(soup.find('section').find_all('strong'))
        except:
            print("Can't get bold text count")
    

    def get_blockquotes(self, soup):
        try:
            notes = []
            blockquotes = soup.find_all('blockquote')
            for blockquote in blockquotes:
                notes.append(blockquote.text)
            return notes
        except:
            print("Couldn't get notes from article.")


    def italic_text_count(self, soup):
        try:
            return len(soup.find('section').find_all('em'))
        except:
            print("Can't get italic text count")

    def count_vids(self, soup):
        try:
            yt_vids = []
            article_soup = soup.find('article')
            for figure in article_soup.find_all('figure'):
                yt_soup = figure.find('iframe', src=re.compile('.*youtube.*'))
                if yt_soup == None:
                    continue
                else:
                    yt_vids.append(yt_soup)
                    
            return len(yt_vids)
                    
        except:
            print("Couldn't get YouTube videos.") 
    

    def count_gists(self, soup):
        try:
            gists = []
            article_soup = soup.find('article')
            for fig in article_soup.find_all('figure'):
                gist_soup = fig.find('iframe', title=re.compile('.*\.py'))
                if gist_soup == None:
                    continue
                else:
                    gists.append(gist_soup)
            return len(gists)
        except:
            print("Couldn't get count of gists.") 


    def count_links(self, soup):
        try:
            links = []
            for a in soup.find('section').find_all('a'):
                link = a.get('href')
                if link != None:
                    links.append(link)
            return len(links)
        except:
            print("Couldn't get amount of links.") 

    def count_code_chunks(self, soup):
        try:
            return len(soup.find_all('pre'))
        except:
            print("Couldn't get amount of code chunks from article.")
    
    def scrape(self, soup, link:str):
        im_size = self.get_mean_size(soup)
        if im_size == None:
            im_size = (None, None)
        article_data = {
            "title": self.get_title(soup),
            "publication": self.get_publication(soup),
            "link": link,
            "author": self.get_author(soup),
            "followers": self.get_followers(soup),
            "reading_time": self.get_reading_time(soup),
            "n_words": self.count_words(soup),
            "pure_text": self.get_pure_text(soup),
            #"blockquotes": self.get_blockquotes(soup),
            "date": self.get_date(soup),
            "responses": self.get_responses(soup),
            "n_code_chunks": self.count_code_chunks(soup),
            "bold_text_count": self.bold_text_count(soup),
            "italic_text_count": self.italic_text_count(soup),
            "mean_image_width": im_size[0],
            "mean_image_height": im_size[1],
            "n_images": self.count_figures(soup),
            "n_lists": self.count_lists(soup),
            #"n_gists": self.count_gists(soup),
            "n_vids": self.count_vids(soup),
            "n_links": self.count_links(soup),
            "claps": self.get_claps(soup)
        }
        '''
        # !'bold_text_count',
        # !'count_code_chunks',
        # !'count_figures',
        # !'count_gists',
        # !'count_links',
        # !'count_lists',
        # !'count_vids',
        # !'count_words',
        # !'get_author',
        # !'get_blockquotes',
        # ! 'get_date',
        # !'get_followers',
        # !'get_mean_size',
        # !'get_publication',
        # !'get_pure_text',
        # !'get_reading_time',
        # !'get_subtitle',
        # !'get_title',
        # !'italic_text_count',
        '''
            
        return(article_data)

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import cv2
from tqdm import tqdm, tqdm_notebook
import lxml
from PIL import Image
from langdetect import detect

In [1]:
from selenium.webdriver.edge.options import Options

options = Options()
# options.add_argument("--headless")
options.add_argument("--incognito")
# options.add_argument("--no-sandbox")
options.add_argument("--window-size=1280,800")

In [14]:
def scrape_page(link, options):
    page_url = link

    driver = webdriver.Edge(options=options)
    try:
        driver.get(page_url)
    except:
        print('Time out')
        return {
            "title": None,
            "publication": None,
            "link": link,
            "author": None,
            "followers": None,
            "reading_time": None,
            "n_words": None,
            "pure_text": None,
            #"blockquotes": self.get_blockquotes(soup),
            "date": None,
            "responses": None,
            "n_code_chunks": None,
            "bold_text_count": None,
            "italic_text_count": None,
            "mean_image_width": None,
            "mean_image_height": None,
            "n_images": None,
            "n_lists": None,
            #"n_gists": self.count_gists(soup),
            "n_vids": None,
            "n_links": None,
            "claps": None
        }, 0
    time.sleep(10)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    article_scrapper = ArticleScraper()
    return article_scrapper.scrape(soup=soup, link=link), soup

In [52]:
url = train_data.loc[0, "link"]

In [53]:
res, soup = scrape_page(url, options)

In [15]:
columns = ['title', 'publication', 'author', 'followers', 
            'reading_time', 'n_words', 'pure_text', 'date', 
            'responses', 'n_code_chunks', 'bold_text_count', 
            'italic_text_count', 'mean_image_width', 'mean_image_height', 
            'n_images', 'n_lists', 'n_vids', 'n_links', 'claps']

In [18]:
columns

['title',
 'publication',
 'author',
 'followers',
 'reading_time',
 'n_words',
 'pure_text',
 'date',
 'responses',
 'n_code_chunks',
 'bold_text_count',
 'italic_text_count',
 'mean_image_width',
 'mean_image_height',
 'n_images',
 'n_lists',
 'n_vids',
 'n_links',
 'claps']

In [19]:
train_data = train_data.rename(columns={"text": "pure_text"})
test_data = test_data.rename(columns={"text": "pure_text"})

In [16]:
another_data = another_data.rename(columns={"url": "link"}).drop(columns=["subtitle", "image"])

In [17]:
def scrap_articles_csv(df: pd.DataFrame, rows: int=50, 
                columns:list=columns, options=options,
                path:str="./scrapped_data/scrapped_data2.csv"):
    if os.path.exists(path):
        df_scrapped = pd.read_csv(path)
    else:
        df_scrapped = pd.DataFrame(columns=columns)
    count = 0
    for i, row in tqdm(df.iterrows(), total=df.shape[0]):
        res, _ = scrape_page(row['link'], options)
        print(count, df.shape[0])
        for col in df.columns:
            if res[col] == None:
                res[col] = row[col]
        df_scrapped.loc[df_scrapped.shape[0], :] = res   
        if (count % rows == 0 or count == df.shape[0] - 1):
            print("Saving to" + path)
            df_scrapped.to_csv(path, index=False)
        count += 1
    return df_scrapped

In [18]:
least_data = another_data.tail(2000)

In [2]:
scrap_articles_csv(least_data)

NameError: name 'scrap_articles_csv' is not defined

In [87]:
scrapped = pd.read_csv("scrapped_data/scrapped_data.csv")

In [96]:
another_data.shape[0] //

650

In [ ]:
np.array_split

## Make Scrapped data become better

In [13]:
data = pd.read_csv('train_noise.csv', index_col='idx')

In [14]:
test_data = pd.read_csv(TRAIN_PATH, index_col='id')

In [15]:
len(test_data) == len(data)

True

In [16]:
data.sample(3)

,title,publication,author,followers,reading_time,n_words,pure_text,date,n_code_chunks,bold_text_count,italic_text_count,mean_image_width,mean_image_height,n_images,n_lists,n_vids,n_links,target
idx,,,,,,,,,,,,,,,,,,
2795,Stats essentials for data science,Towards Data Science,Matt Sosna,365.0,22.0,4815.0,Whether you’re spending all day in spreadsheet...,06/05/2021,0.0,54.0,159.0,1210.0,582.0,25.0,6.0,0.0,75.0,231.0
2410,What I Learned by Ditching College to Travel a...,Mission.org,Jake Heilbrunn,205.0,8.0,1581.0,“The real voyage of discovery consists not in ...,28/09/2016,0.0,5.0,11.0,1019.0,794.0,11.0,0.0,0.0,13.0,133.0
2834,Illustrated Guide to LSTM’s and GRU’s: A step ...,Towards Data Science,Michael Phi,3800.0,10.0,2003.0,Hi and welcome to an Illustrated Guide to Long...,24/09/2018,0.0,6.0,6.0,NaN,NaN,19.0,0.0,0.0,8.0,20000.0


In [17]:
is_null = data.isnull()
row_with_null = is_null.all(axis=1)
rows_with_null = data[row_with_null].index
with tqdm(rows_with_null, unit="page") as tepoch:
    for idx in tepoch:
        new_row = scrape_page(test_data.loc[idx].link, options)
        data.loc[idx,
                data.columns] = new_row

0page [00:00, ?page/s]


In [18]:
def scrape_column(column):
    print('Before repeated scrapping: ', data[column].isna().sum())
    indeces = data[data[column].isna() == True].index
    with tqdm(indeces, unit="page") as tepoch:
        for idx in tepoch:
            new_row = scrape_page(test_data.loc[idx].link, options)
    #     print(new_row['pure_text'])
            data.loc[idx,
                column] = new_row[column]
    print('After scrapping: ', data[column].isna().sum())

In [19]:
scrape_column('title')

Before repeated scrapping:  43


  0%|          | 0/43 [00:00<?, ?page/s]

Can't get image's sizes
Couldn't get title from article.
Couldn't get title from article.
Can't go to the followers info
Couldn't get reading time from article.
Couldn't get date from article.
Couldn't get YouTube videos.


  5%|▍         | 2/43 [00:27<09:20, 13.67s/page]

Couldn't get title from article.


...

Couldn't get title from article.


 98%|█████████▊| 42/43 [07:58<00:11, 11.31s/page]

Can't get image's sizes
Couldn't get title from article.
Couldn't get title from article.
Can't go to the followers info
Couldn't get reading time from article.
Couldn't get date from article.
Couldn't get YouTube videos.


100%|██████████| 43/43 [08:06<00:00, 11.31s/page]

After scrapping:  35


In [55]:
indeces_title = data[data['title'].isna() == True].index

In [56]:
test_data.loc[indeces_title].head(2)

,author,reading_time,link,title,text,claps
idx,,,,,,
156,Bartosz Szabłowski,19,https://towardsdatascience.com/how-convolution...,How Convolutional Neural Network works | by Ba...,"Towards Data Science\nNov 15, 2020\nOne of the...",149.0
331,Sam Taha,5,https://medium.com/@tahasam/bots-ai-and-the-fu...,"Bots, AI and the Future of Augmented UX Design...","Jan 1, 2000\nBots, AI and the Future of Augmen...",0.0


In [57]:
data.loc[indeces_title].head(2)

,title,publication,author,followers,reading_time,n_words,pure_text,date,n_code_chunks,bold_text_count,italic_text_count,mean_image_width,mean_image_height,n_images,n_lists,n_vids,n_links,target
idx,,,,,,,,,,,,,,,,,,
156,How Convolutional Neural Network works,Towards Data Science,Bartosz Szabłowski,NaN,19.0,240.0,The first issue of image analysis is classific...,15/11/2020,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,149.0
331,"Bots, AI and the Future of Augmented UX Design",NaN,Sam Taha,27.0,5.0,233.0,"Now, what if we could have the same augmented ...",01/01/2000,0.0,1.0,3.0,380.0,225.0,4.0,0.0,0.0,1.0,0.0


In [43]:
for idx in [156, 545, 828, 905, 1553, 1973, 2674, 3455]:
    date_string = test_data.loc[idx].text.split('\n')[1]    
    data.iat[idx, data.columns.get_loc('date')] = datetime.datetime.strptime(date_string, '%b %d, %Y').strftime('%d/%m/%Y') 
    data.iat[idx, data.columns.get_loc('publication')] = test_data.loc[idx].text.split('\n')[0] 


In [36]:
date_indeces = set(indeces_title) - set([156, 545, 828, 905, 1553, 1973, 2674, 3455])

In [40]:
import datetime
for idx in date_indeces:
    date_string = test_data.loc[idx].text.split('\n')[0]    
    data.iat[idx, data.columns.get_loc('date')] = datetime.datetime.strptime(date_string, '%b %d, %Y').strftime('%d/%m/%Y') 
# publication [156, 545, 828, 905, 1553, 1973, 2674, 3455]

In [49]:
test_data.loc[indeces_title].head(2)

,author,reading_time,link,title,text,claps
idx,,,,,,
156,Bartosz Szabłowski,19,https://towardsdatascience.com/how-convolution...,How Convolutional Neural Network works | by Ba...,"Towards Data Science\nNov 15, 2020\nOne of the...",149.0
331,Sam Taha,5,https://medium.com/@tahasam/bots-ai-and-the-fu...,"Bots, AI and the Future of Augmented UX Design...","Jan 1, 2000\nBots, AI and the Future of Augmen...",0.0


In [54]:
for idx in indeces_title:
    data.iat[idx, data.columns.get_loc('title')]  = test_data.loc[idx].title.split('|')[0]
    data.iat[idx, data.columns.get_loc('author')]  = test_data.loc[idx].author
    data.iat[idx, data.columns.get_loc('reading_time')]  = test_data.loc[idx].reading_time
    data.iat[idx, data.columns.get_loc('pure_text')] = test_data.loc[idx].text.split('\n')[3]
    data.iat[idx, data.columns.get_loc('n_words')] = len(test_data.loc[idx].text.split('\n')[3].split())
#author, title, reading_time

In [67]:
data.loc[indeces_title].head(1)

,title,publication,author,followers,reading_time,n_words,pure_text,date,n_code_chunks,bold_text_count,italic_text_count,mean_image_width,mean_image_height,n_images,n_lists,n_vids,n_links,target
idx,,,,,,,,,,,,,,,,,,
156,How Convolutional Neural Network works,Towards Data Science,Bartosz Szabłowski,NaN,19.0,240.0,The first issue of image analysis is classific...,15/11/2020,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,149.0


In [61]:
data['title'].isna().sum()

0

In [62]:
scrape_column('publication')

Before repeated scrapping:  944


  2%|▏         | 15/944 [03:11<3:03:49, 11.87s/page]

Can't get image's sizes


  3%|▎         | 24/944 [04:58<3:14:53, 12.71s/page]

Can't get image's sizes


...

Can't get image's sizes


100%|█████████▉| 941/944 [3:11:12<00:32, 10.99s/page]

Can't go to the followers info


100%|██████████| 944/944 [3:11:45<00:00, 12.19s/page]

After scrapping:  936


In [69]:
data.loc[data[data['publication'].isna() == True].index]['author']

idx
7               Adam Simek
18           John Mannelly
23           Sagar Pokhrel
29              Ratul Aich
31            Matthew Chan
               ...        
3736       napat thumvanit
3737    Aleksi Pietikäinen
3738            Meghakumar
3746          Adam Pickard
3755         Neha Tripathi
Name: author, Length: 936, dtype: object

In [74]:
MONTH = ['Jan', 'Feb', 'Mar', 'Apr',
            'May', 'Jun', 'Jul', 'Aug',
            'Sep', 'Oct', 'Nov', 'Dec']

In [95]:
orgs = []
for text, idx in zip(
    test_data.loc[data[data['publication'].isna() == True].index].text.values,
    data[data['publication'].isna() == True].index):
    text_ = text.split('\n')
    if text.split('\n')[0].split(' ')[0] in MONTH:
        continue
    else:
        print(text.split('\n')[0])

Machine Intelligence Report
ML Review
Machine Intelligence Report


In [295]:
data.publication = data.publication.fillna('Medium').values

In [296]:
data['publication'].isna().sum()

0

In [63]:
scrape_column('author')

Before repeated scrapping:  11


 36%|███▋      | 4/11 [00:44<01:16, 10.97s/page]

Can't get image's sizes


100%|██████████| 11/11 [02:10<00:00, 11.88s/page]

After scrapping:  0


In [64]:
scrape_column('followers') #0, 9, 10, 24, 35, 43, 44

Before repeated scrapping:  56


  0%|          | 0/56 [00:00<?, ?page/s]

Can't get image's sizes
Couldn't get title from article.
Couldn't get title from article.
Can't go to the followers info
Couldn't get reading time from article.
Couldn't get date from article.
Couldn't get YouTube videos.


  2%|▏         | 1/56 [00:08<07:42,  8.41s/page]

Can't go to the followers info


...

Can't go to the followers info


 98%|█████████▊| 55/56 [11:23<00:09,  9.24s/page]

Can't go to the followers info


100%|██████████| 56/56 [11:35<00:00, 12.42s/page]

After scrapping:  45


In [124]:
indeces_followers = data[data['followers'].isna() == True].index

In [135]:
mean_fol_idx = [0, 9, 42]
# * 0
# * 9,
# * 42

In [160]:
mean_fol = []
mean_fol.append(np.mean(data[data['publication'] == data.loc[indeces_followers[0]]['publication']][data['followers'].isna() == False]['followers']))
mean_fol.append(np.mean(data[data['followers'].isna() == False]['followers']))
mean_fol.append(np.mean(data[data['followers'].isna() == False]['followers']))
mean_fol

C:\Users\Aleksey\AppData\Local\Temp\ipykernel_81300\7504647.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  mean_fol.append(np.mean(data[data['publication'] == data.loc[indeces_followers[0]]['publication']][data['followers'].isna() == False]['followers']))


[2258.32333767927, 4019.5602263540823, 4019.5602263540823]

In [161]:
mean_fol = [
    int(i) for i in mean_fol]

In [164]:
data.loc[indeces_followers[mean_fol_idx], 'followers'] = mean_fol

In [165]:
indeces_followers = data[data['followers'].isna() == True].index
len(indeces_followers)

42

In [166]:
for idx in indeces_followers:
    data.loc[idx,
                'followers'] = 1

In [167]:
data.followers.isna().sum()

0

In [169]:
scrape_column('reading_time')

Before repeated scrapping:  0


0page [00:00, ?page/s]

After scrapping:  0


In [170]:
scrape_column('n_words')

Before repeated scrapping:  0


0page [00:00, ?page/s]

After scrapping:  0


In [171]:
scrape_column('pure_text')

Before repeated scrapping:  0


0page [00:00, ?page/s]

After scrapping:  0


In [172]:
scrape_column('date')

Before repeated scrapping:  0


0page [00:00, ?page/s]

After scrapping:  0


In [105]:
indeces_date = data[data['date'].isna() == True].index
indeces_date

Int64Index([2857, 2990, 3645], dtype='int64', name='idx')

In [109]:
dates = [date.split('\n')[0] +', 2022' for date in test_data.loc[indeces_date].text.values[:-1]]

In [113]:
dates += [date.split('\n')[1] +', 2022' for date in [test_data.loc[indeces_date].text.values[-1]]]

In [115]:
dates = [datetime.datetime.strptime(date_string, '%b %d, %Y').strftime('%d/%m/%Y') for date_string in dates]

In [119]:
data.loc[indeces_date, 'date'] = dates

In [168]:
data.date.isna().sum()

0

In [173]:
scrape_column('n_code_chunks')

Before repeated scrapping:  0


0page [00:00, ?page/s]

After scrapping:  0


In [174]:
scrape_column('bold_text_count')

Before repeated scrapping:  0


0page [00:00, ?page/s]

After scrapping:  0


In [175]:
scrape_column('italic_text_count')

Before repeated scrapping:  0


0page [00:00, ?page/s]

After scrapping:  0


In [176]:
scrape_column('n_images')

Before repeated scrapping:  0


0page [00:00, ?page/s]

After scrapping:  0


In [177]:
data.loc[data[data['n_images'] == 0].index, ['mean_image_width', 'mean_image_height']] = [0, 0]

In [178]:
data.mean_image_width.isna().sum(), data.mean_image_height.isna().sum()

(125, 125)

In [179]:
indeces_width = data[data['mean_image_width'].isna() == True].index

In [180]:
data.loc[indeces_width, 'mean_image_width'] = int(np.mean(data.mean_image_width))

In [181]:
indeces_height = data[data['mean_image_height'].isna() == True].index

In [182]:
data.loc[indeces_height, 'mean_image_height'] = int(np.mean(data.mean_image_height))

In [183]:
data.mean_image_width.isna().sum(), data.mean_image_height.isna().sum()

(0, 0)

In [184]:
scrape_column('n_images')

Before repeated scrapping:  0


0page [00:00, ?page/s]

After scrapping:  0


In [185]:
scrape_column('n_lists')

Before repeated scrapping:  8


100%|██████████| 8/8 [02:00<00:00, 15.05s/page]

After scrapping:  0


In [186]:
scrape_column('n_vids')

Before repeated scrapping:  19


  0%|          | 0/19 [00:00<?, ?page/s]

Can't get image's sizes
Couldn't get title from article.
Couldn't get title from article.
Can't go to the followers info
Couldn't get reading time from article.
Couldn't get date from article.
Couldn't get YouTube videos.


 26%|██▋       | 5/19 [00:53<02:33, 10.93s/page]

Can't get image's sizes


...

Can't get image's sizes
Couldn't get title from article.
Couldn't get title from article.
Can't go to the followers info
Couldn't get reading time from article.
Couldn't get date from article.
Couldn't get YouTube videos.


 95%|█████████▍| 18/19 [03:16<00:10, 10.11s/page]

Can't get image's sizes
Couldn't get title from article.
Couldn't get title from article.
Can't go to the followers info
Couldn't get reading time from article.
Couldn't get date from article.
Couldn't get YouTube videos.


100%|██████████| 19/19 [03:24<00:00, 10.74s/page]

After scrapping:  7


In [190]:
data.loc[data[data['n_vids'].isna() == True].index, 'n_vids'] = 0

In [191]:
data.n_vids.isna().sum()

0

In [187]:
scrape_column('n_links')

Before repeated scrapping:  8


100%|██████████| 8/8 [02:10<00:00, 16.29s/page]

After scrapping:  0


In [218]:
languages = []

for text in data.pure_text:
    try:
        languages.append(detect(text[:50]))
    except:
        print('Sorry')
        print(text[:50])
        languages.append(None)

Sorry

Sorry
__________________________________________________
Sorry

Sorry
https://sarit-maitra.medium.com/membershipAnomaly 
Sorry
http://www.youtube.com/watch?v=fMQ8u6MSgF4Galvatro


In [219]:
data['language'] = languages

In [227]:
texts = test_data.loc[data[data.language.isna() == True].index].text.values

In [271]:
to_fil = [texts[0].split('\n')[5]]

In [272]:
to_fil += [' '.join(texts[1].split('\n')[2:-9])]

In [273]:
to_fil += [texts[2].split('\n')[5]]

In [274]:
to_fil += [' '.join(texts[3].split('\n')[2:4])]

In [275]:
to_fil += [' '.join(texts[4].split('\n')[2:5])]

In [280]:
data.loc[data[data.language.isna() == True].index, 'pure_text'] = to_fil

In [281]:
languages = []

for text in data.pure_text:
    try:
        languages.append(detect(text[:50]))
    except:
        print('Sorry')
        print(text[:50])
        languages.append(None)

Sorry
http://www.youtube.com/watch?v=fMQ8u6MSgF4Galvatro


In [283]:
data['language'] = languages

In [289]:
data[data.language.isna() == True].index

Int64Index([3423], dtype='int64', name='idx')

In [292]:
data.loc[3423, 'language'] = 'en'

In [300]:
for column in data.columns:
    print(column)
    scrape_column(column)

title
Before repeated scrapping:  0


0page [00:00, ?page/s]


After scrapping:  0
publication
Before repeated scrapping:  0


0page [00:00, ?page/s]


After scrapping:  0
author
Before repeated scrapping:  0


...

After scrapping:  0
language
Before repeated scrapping:  0


0page [00:00, ?page/s]


After scrapping:  0
(3423, 'language')
Before repeated scrapping:  0


0page [00:00, ?page/s]

After scrapping:  0


In [306]:
len(data) == len(test_data)

True

In [309]:
# data.to_csv(os.path.join(SCRAPPING_OUT_PATH, 'scrapped_test.csv'))
data.to_csv(os.path.join(SCRAPPING_OUT_PATH, 'scrapped_train.csv'))